# Import Data

In [1]:
from utils.data_preprocess import *
from utils.tf_helpers import create_model_checkpoint, evaluate_preds
import pandas as pd
import numpy as np

In [20]:
df = pd.read_csv('data\ETHBTC-5m-data.csv')
df = df.set_index('Timestamp')
df = df.loc[:, ['Close']]
df.head()

,Close
Timestamp,
2017-07-14 04:00:00,0.080000
2017-07-14 04:05:00,0.080001
2017-07-14 04:10:00,0.086400
2017-07-14 04:15:00,0.085128
2017-07-14 04:20:00,0.086000


## Turn Dataset into tf.dataset to Train Faster

In [21]:
datas = data_split(df, 3)
train_df, val_df, test_df = train_val_test_split(datas[0])

In [22]:
Data = DataPreprocess(train_df, val_df, test_df, window_size=24, label_size=1, shift=1, label_columns=['Close'])
Data

Total window size: 24
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [24]
Label column name(s): ['Close']

In [23]:
train_ds = Data.train.prefetch(tf.data.AUTOTUNE)
val_ds = Data.val.prefetch(tf.data.AUTOTUNE)
test_ds = Data.test.prefetch(tf.data.AUTOTUNE)

# Build an MLP Model

In [24]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential

In [26]:
def MLP_model():
    model = Sequential([
        layers.Dense(128, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='linear')
    ], name='MLP_ohlcv')
    
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    return model

model = MLP_model()

In [27]:
model.fit(train_ds,
          epochs=2000,
          steps_per_epoch=len(train_ds),
          validation_data=val_ds,
          validation_steps=len(val_ds),
          callbacks=[create_model_checkpoint(model_name=model.name),
                     tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1),
                     tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)])

Epoch 1/2000
827/827 [==============================] - 9s 10ms/step - loss: 3.6893e-05 - mae: 0.0030 - val_loss: 6.3816e-04 - val_mae: 0.0236
INFO:tensorflow:Assets written to: model_experiments\MLP_ohlcv\assets
Epoch 2/2000
827/827 [==============================] - 8s 9ms/step - loss: 1.8489e-05 - mae: 0.0031 - val_loss: 9.6249e-04 - val_mae: 0.0293
Epoch 3/2000
827/827 [==============================] - 8s 9ms/step - loss: 2.4746e-05 - mae: 0.0037 - val_loss: 9.8733e-04 - val_mae: 0.0297
Epoch 4/2000
827/827 [==============================] - 8s 10ms/step - loss: 2.6494e-05 - mae: 0.0040 - val_loss: 9.7709e-04 - val_mae: 0.0295
Epoch 5/2000
827/827 [==============================] - 8s 10ms/step - loss: 2.6652e-05 - mae: 0.0039 - val_loss: 9.0112e-04 - val_mae: 0.0282
Epoch 6/2000
827/827 [==============================] - 8s 10ms/step - loss: 2.9373e-05 - mae: 0.0040 - val_loss: 8.9697e-04 - val_mae: 0.0281

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.00050000002374

In [38]:
model.evaluate(val_ds)

237/237 [==============================] - 2s 7ms/step - loss: 6.3816e-04 - mae: 0.0236


[0.0006381588755175471, 0.023607874289155006]

In [30]:
# Create a function to predict the future
def make_future_forecasts(values, model, into_future, window_size=24) -> list:
    """
    Make future forecasts into_future steps after values ends.

    Returns future forecasts as a list of floats
    """
    # Create an empty list for future forecasts/prepare data to forecast on
    future_forecasts = []
    last_window = values[-window_size:]
    # Make INTO_FUTURE number of predictions, altering the data which gets predicted on each
    for _ in range(into_future):
        # Predict on the last window then append it again
        future_pred = model.predict(tf.expand_dims(last_window, axis=0))
        print(f'Predicting on: \n {last_window} -> Prediction: {tf.squeeze(future_pred).numpy()}\n')
        # Append predictions to future forecast
        future_forecasts.append(tf.squeeze(future_pred).numpy())
        # Update last window with new pred and get WINDOW_SIZE most recent preds (model was trained on WINDOW_SIZE window)
        last_window = np.append(last_window, future_pred)[-window_size:]
    return future_forecasts